In [1]:
import os
import yaml
from torch.utils.data import DataLoader
import argparse

from GeospatialFM.data import get_datasets
from GeospatialFM.models import *
# from utils import load_config
from torchgeo.samplers import RandomGeoSampler
from matplotlib import pyplot as plt

from transformers import TrainingArguments, Trainer
from transformers import AdamW, get_linear_schedule_with_warmup
from GeospatialFM.utils import setup, get_eval_fn
from GeospatialFM.data import *

from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
import numpy as np
from torch.utils.data import ConcatDataset
import segmentation_models_pytorch as smp

%load_ext autoreload
%autoreload 2

/home/haozhesi/anaconda3/envs/sat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def extract_features(model, dataloader, device):
    x_all = []
    y_all = []

    for batch in tqdm(dataloader):
        images = batch["image"].to(device)
        labels = batch["label"].numpy()
        
        with torch.inference_mode():
            features = model(images).cpu().numpy()
        
        x_all.append(features)
        y_all.append(labels)

    x_all = np.concatenate(x_all, axis=0)
    y_all = np.concatenate(y_all, axis=0)

    return x_all, y_all

In [2]:
device = torch.device('cuda:0')

In [3]:
args = {'exp_name': None,
        'config_file': 'GeospatialFM/configs/bigearthnet/bn_rn50_dino.yaml',
        'opts': None, 
        'save_config': False}
args = argparse.Namespace(**args)
args.debug = True
args

Namespace(exp_name=None, config_file='GeospatialFM/configs/bigearthnet/bn_rn50_dino.yaml', opts=None, save_config=False, debug=True)

In [4]:
cfg, _ = setup(args)

In [9]:
train_ds, val_ds, test_ds = get_datasets(cfg['DATASET'])
model = construct_model(cfg['MODEL'])
model = model.to(device)
len(train_ds), len(val_ds), len(test_ds)

(39341, 125866, 125866)

In [10]:
train_dl = DataLoader(train_ds, batch_size=512, shuffle=True, num_workers=8)
test_dl = DataLoader(test_ds, batch_size=512, shuffle=False, num_workers=8)

In [11]:
x_all, y_all = extract_features(model.base_model, train_dl, device)

  0%|          | 0/77 [00:22<?, ?it/s]


KeyboardInterrupt: 

In [1]:
linear_model = LogisticRegression(C=50.0, max_iter=1000)
linear_model.fit(x_all, y_all)

NameError: name 'LogisticRegression' is not defined

In [ ]:
linear_model.score(x_test, y_test)